# UNESCO Heritage Sites Risk Modeling - Visualization

## Notebook 03: Interactive Visualizations and Maps

**Purpose**: Create interactive visualizations and export publication-quality figures.

**Contents**:
1. Generate interactive Folium maps
2. Create radar charts for risk factors
3. Export publication-quality figures
4. Dashboard-ready visualizations

In [ ]:
# Import required libraries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap, MarkerCluster
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path

import sys
sys.path.append('..')
from src.db.connection import engine

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ Libraries imported")

## 1. Load Data

In [ ]:
# Load risk data
query = """
SELECT 
    hs.name, hs.country, hs.category,
    ST_X(hs.geom) as longitude, ST_Y(hs.geom) as latitude,
    rs.urban_density_score, rs.climate_anomaly_score,
    rs.seismic_risk_score, rs.fire_risk_score,
    rs.flood_risk_score, rs.coastal_risk_score,
    rs.composite_risk_score, rs.risk_level, rs.is_anomaly
FROM unesco_risk.heritage_sites hs
JOIN unesco_risk.risk_scores rs ON hs.id = rs.site_id;
"""

df = pd.read_sql(query, engine)
print(f"Loaded {len(df)} sites")

## 2. Interactive Folium Map

In [ ]:
# Create base map
m = folium.Map(
    location=[50, 10],
    zoom_start=4,
    tiles='CartoDB positron'
)

# Color mapping
risk_colors = {
    'low': 'green',
    'medium': 'orange',
    'high': 'red',
    'critical': 'darkred'
}

# Add markers
for _, row in df.iterrows():
    color = risk_colors.get(row['risk_level'], 'gray')
    
    popup_html = f"""
    <b>{row['name']}</b><br>
    Country: {row['country']}<br>
    Risk Level: {row['risk_level']}<br>
    Composite Score: {row['composite_risk_score']:.3f}
    """
    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5 if not row['is_anomaly'] else 8,
        color=color,
        fill=True,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(m)

# Save map
output_path = Path('../output/maps/notebook_risk_map.html')
output_path.parent.mkdir(parents=True, exist_ok=True)
m.save(str(output_path))

print(f"Map saved to {output_path}")
m

## 3. Radar Chart for Top Risk Sites

In [ ]:
# Select top 5 highest risk sites
top_sites = df.nlargest(5, 'composite_risk_score')

categories = ['Urban', 'Climate', 'Seismic', 'Fire', 'Flood', 'Coastal']
score_cols = [
    'urban_density_score', 'climate_anomaly_score',
    'seismic_risk_score', 'fire_risk_score',
    'flood_risk_score', 'coastal_risk_score'
]

# Create radar chart
fig = go.Figure()

for _, site in top_sites.iterrows():
    values = [site[col] for col in score_cols]
    values.append(values[0])  # Close the loop
    
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=categories + [categories[0]],
        name=site['name'][:30],
        fill='toself'
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(visible=True, range=[0, 1])
    ),
    title='Risk Factor Comparison - Top 5 Sites',
    showlegend=True
)

fig.show()

## 4. Risk Score Treemap by Country

In [ ]:
# Aggregate by country
country_summary = df.groupby('country').agg({
    'composite_risk_score': 'mean',
    'name': 'count'
}).reset_index()
country_summary.columns = ['country', 'avg_risk', 'site_count']

# Filter countries with at least 2 sites
country_summary = country_summary[country_summary['site_count'] >= 2]

# Create treemap
fig = px.treemap(
    country_summary,
    path=['country'],
    values='site_count',
    color='avg_risk',
    color_continuous_scale='RdYlGn_r',
    title='Sites per Country (colored by average risk)'
)

fig.show()

## 5. Export High-Quality Figures

In [ ]:
# Risk distribution plot for publication
fig, ax = plt.subplots(figsize=(10, 6), dpi=300)

risk_order = ['low', 'medium', 'high', 'critical']
colors_map = {'low': '#2ecc71', 'medium': '#f39c12', 'high': '#e74c3c', 'critical': '#c0392b'}

counts = df['risk_level'].value_counts().reindex(risk_order, fill_value=0)
bars = ax.bar(range(len(counts)), counts.values,
              color=[colors_map[level] for level in counts.index],
              edgecolor='black', linewidth=1.5)

ax.set_xticks(range(len(counts)))
ax.set_xticklabels(counts.index.str.capitalize(), fontsize=12)
ax.set_ylabel('Number of Sites', fontsize=12, fontweight='bold')
ax.set_title('UNESCO Heritage Sites Risk Distribution',
             fontsize=14, fontweight='bold', pad=20)
ax.grid(True, axis='y', alpha=0.3, linestyle='--')
ax.spines['top'].set_visible(False)
ax.spines('right'].set_visible(False)

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

# Save
output_dir = Path('../output/figures')
output_dir.mkdir(parents=True, exist_ok=True)
plt.savefig(output_dir / 'risk_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Figure saved to {output_dir / 'risk_distribution.png'}")

## Summary

This notebook created:
1. Interactive Folium map of all sites
2. Radar chart comparing risk factors
3. Treemap visualization by country
4. Publication-quality figures

**Output files**:
- `output/maps/notebook_risk_map.html`
- `output/figures/risk_distribution.png`

For the full interactive dashboard, run:
```bash
python run_dashboard.py
```